In [1]:
import numpy as np 
import pandas as pd
import torch
from sklearn.metrics import r2_score
from pyro import clear_param_store
import pyro as pyro
import pyro.contrib.gp as gp
from pyro.nn import PyroSample
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS, Predictive,HMC
import arviz as az
from scipy.optimize import curve_fit


# Importing data 

In [2]:
PM25=pd.read_excel("../data/Pre_flow_cal.xlsx")
train,test=PM25.loc[PM25.label==0,:],PM25.loc[PM25.label==1,:]

# Converting  PM25 data to tensors and put on GPU

In [3]:
# put data on cuda else cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#make train and test data ready for flow of pump vs pm2.5 ready 
X_pm25=torch.tensor(train.flow.values).float().to(device)
y_pm25=torch.tensor(train.pm_final.values).float().to(device)
X_pm25_test=torch.tensor(test.flow.values).float().to(device)
y_pm25_test=test.pm_final.values

In [4]:
clear_param_store();
rbf = gp.kernels.RBF(input_dim=1);

rbf.variance = PyroSample(dist.HalfNormal(torch.tensor(PM25.pm_final.mean())))

rbf.lengthscale = PyroSample(dist.HalfNormal(torch.tensor(2.)))
gpr = gp.models.GPRegression(X_pm25,y_pm25, rbf).to(device);
gpr.noise = PyroSample(dist.HalfNormal(torch.tensor(20.)))

nuts_kernel = NUTS(gpr.model);

mcmc = MCMC(nuts_kernel, num_samples=1000,warmup_steps=9000,num_chains=1);

mcmc.run();

Sample: 100%|██████████████████████████████████████| 10000/10000 [02:58, 55.93it/s, step size=7.60e-01, acc. prob=0.907]


In [5]:
torch.save(gpr, "../models/PM2.5_flow");

In [6]:
posterior_samples = mcmc.get_samples();
posterior_predictive = Predictive(gpr, posterior_samples)(X_pm25);
prior = Predictive(gpr, num_samples=500)(X_pm25);

pyro_data = az.from_pyro(mcmc,
    prior=prior,
    posterior_predictive=posterior_predictive

);
az.to_json(pyro_data, "../Arviz_stats/mcmc_pm25_flow.json")

/data/michaelf/miniconda3/envs/gaussion_process/lib/python3.11/site-packages/arviz/data/io_pyro.py:158: UserWarning: Could not get vectorized trace, log_likelihood group will be omitted. Check your model vectorization or set log_likelihood=False
  warnings.warn(


'../Arviz_stats/mcmc_pm25_flow.json'

In [7]:
az.style.use("arviz-doc")

ax = az.plot_trace(pyro_data,backend='bokeh')

In [8]:
X_bc=torch.tensor(train.flow.values).float().to(device)
y_bc=torch.tensor(train.sv_final.values).float().to(device)
X_bc_test=torch.tensor(test.flow.values).float().to(device)
y_bc_test=test.sv_final.values

In [9]:
clear_param_store()
rbf_bc = gp.kernels.RBF(input_dim=1)

rbf_bc.variance = PyroSample(dist.HalfNormal(torch.tensor(PM25.sv_final.mean())))

rbf_bc.lengthscale = PyroSample(dist.HalfNormal(torch.tensor(2.)))
gpr_bc = gp.models.GPRegression(X_bc,y_bc, rbf_bc).to(device)
gpr_bc.noise = PyroSample(dist.HalfNormal(torch.tensor(2.)))


nuts_kernel_bc = NUTS(gpr_bc.model,)


mcmc_bc = MCMC(nuts_kernel_bc,warmup_steps=9000, num_samples=1000,num_chains=1)

mcmc_bc.run()

Sample: 100%|██████████████████████████████████████| 10000/10000 [03:30, 47.42it/s, step size=5.48e-01, acc. prob=0.944]


In [10]:
torch.save(gpr_bc, "../models/bc_flow");

In [12]:
posterior_samples_bc = mcmc_bc.get_samples(500)
posterior_predictive_bc = Predictive(gpr_bc, posterior_samples_bc)(X_bc)
prior_bc = Predictive(gpr_bc, num_samples=500)(X_bc)

pyro_data_bc = az.from_pyro(mcmc_bc,
    prior=prior_bc,
    posterior_predictive=posterior_predictive_bc,

)
az.to_json(pyro_data_bc, "../Arviz_stats/mcmc_bc_flow.json")

'../Arviz_stats/mcmc_bc_flow.json'